# parameters

In [16]:
#infile='/Users/nikett/quick/sciencesrl-annotation-afresh/results/sandbox_v2_sample.pkl'
# infile='/Users/nikett/quick/sciencesrl-annotation-afresh/results/sandbox_v2_sample.pkl'
#infile ='/Users/nikett/quick/sciencesrl-annotation-afresh/results/batch_v2_8k_partial_v2.pkl'
# infile='/Users/nikett/quick/sciencesrl-annotation-afresh/results/batch_v5_3.pkl'
infile='/Users/nikett/quick/sciencesrl-annotation-afresh/results/batch_v5_3__round3.pkl'
outfile=infile.replace('.pkl','.tsv')
outfile_highconf=infile.replace('.pkl','.highconf.tsv')

In [17]:
from pprint import pprint
import json
from mturk_utils.mturk import pickle_this, unpickle_this
from tqdm import tqdm

def sanitize_span(span):
    return '-'.join([str.strip(str(i)) for i in span.replace('[','').replace(']','').split(',')])

def create_result_scisrl(assmt):
      parsed_json = json.loads(assmt.answers[0][0].fields[0])
      span_sep="#"
      ar_dict={}
          
      try:
        dummy_result = parsed_json["senttokens"]
      except:
        print "Exception in the parsed json response for sentence with response:"
        pprint(parsed_json)
        return {}
    
      sent_id = parsed_json["sentid"]
      tokens = ' '.join([str(t) for t in parsed_json["senttokens"]]) if "senttokens" in parsed_json else "" 
      if "actionRelations" in parsed_json:
              for ar in parsed_json["actionRelations"]:
                ar_dict[str(ar["source"]) +"\t"+ ar["relation"]] = str(ar["target"])
      overall_results = {}
      for verbid,p in enumerate(parsed_json["actions"]):
#       print("==========================")
#       print("sentid: "+str(sent_id) +"\tverbid: "+ str(verbid))
        ar_roles = {}
        ar_roles['sentid']=sent_id
        ar_roles['senttokens']=tokens
        ar_roles['verbid']=verbid
        ar_roles['assignment_id']=assmt.AssignmentId
        ar_roles['hit_id']=assmt.HITId
        ar_roles['worker_id']=assmt.WorkerId
#       print("verb"+"\t"+p['verb']["lemma"]+span_sep+sanitize_span(str(p['verb']["span"])))
        ar_roles['verb']=str(p['verb']["lemma"])+span_sep+sanitize_span(str(p['verb']["span"]))
        for role in ['who','what', 'when','where','whereFrom','whereTo','input','output','enable','prevent']:
            ar_role = str(verbid)+"\t"+ role
            if (ar_role in ar_dict):
                # event-event
                ar_roles[role]=parsed_json["actions"][int(ar_dict[ar_role])]['verb']['lemma']+span_sep+ sanitize_span(str(parsed_json["actions"][int(ar_dict[ar_role])]['verb']['span']))
#               print(role+"\t"+ parsed_json["actions"][int(ar_dict[ar_role])]['verb']['lemma']+span_sep+ sanitize_span(str(parsed_json["actions"][int(ar_dict[ar_role])]['verb']['span'])))
            elif (role in p['questions'] and p['questions'][role]):
                # event-args
                stri = str(p['questions'][role]["phrase"])
                if len(stri) > 1:  # answer must have a minimum length of 2.
                    ar_roles[role]=stri+span_sep+sanitize_span(str(p['questions'][role]["span"]))
                else: 
                    ar_roles[role]=""    
#               print(role+"\t"+p['questions'][role]["phrase"])+span_sep+sanitize_span(str(p['questions'][role]["span"]))
            else:
                # empty answers
                ar_roles[role]=""
#               print(role+"\t"+"")
        overall_results[str(sent_id) +"\t"+ str(verbid)] = ar_roles
      return overall_results
                
turk_data = unpickle_this(infile)
assignments = [item for sublist in turk_data.values() for item in sublist]
# FIXME sent_id cannot be recovered from the json response-- needs to be updated with sentid?
# FIXME need to have a refresh button per verb.
# FIXME common output for all the results.
assignment_results = [create_result_scisrl(ar) for ar in tqdm(assignments)]
role_list=['assignment_id','hit_id','worker_id','sentid','senttokens','verbid','verb', 'who',
           'what', 'when','where','whereFrom','whereTo','input','output','enable','prevent']
out_handle = open(outfile, 'w')
out_handle.write('\t'.join(role_list) +"\n")
for one_result in tqdm(assignment_results):
    for k,v in one_result.items():
        results = [str(v[role]) for role in role_list]
        out_handle.write('\t'.join(results) +"\n")
# print(assignment_results)
out_handle.close()
print "Output is in ", outfile

100%|██████████| 4680/4680 [00:00<00:00, 20679.86it/s]

Output is in  /Users/nikett/quick/sciencesrl-annotation-afresh/results/batch_v5_3__round3.tsv


In [18]:
import pandas as pd
from tqdm import tqdm

def elem_from_countmap(e):
    # FIXME not sure if we can invoke r1's key (without count) without serializing it.
    # the bulb#8-10 spaces 1
    return str(e).split('  ')[0] if e else ""

def longer_overlapping_span(c1,c2):
    # 'the battery#14-16', 'from the battery#13-16'
    # 'the base of the bulb#5-10', 'the base#5-7'
    # 'the base#5-7', 'the base of the bulb#5-10'
    min1, max1 = c1.split('#')[-1].split('-')
    min2, max2 = c2.split('#')[-1].split('-')
    if int(min1)<=int(min2) and int(max1)>=int(max2):
        return c1
    elif int(min2)<=int(min1) and int(max2)>=int(max1):
        return c2
    else:
        return ""

def longest_overlapping_span(confusions):
    # Get the first found subsuming/larger span.
    for c1 in confusions:
        for c2 in confusions:
            if c1!=c2:
                result = longer_overlapping_span(c1,c2)
                if result:
                    return result
    return ""

def high_agreement_during_tie(elems):
    #    case 2.1:  the bulb#8-10, , bulb#9-10  [30QQTY5GMKZ9ES6DB76RSS8NBCCU7A]
    #    case 2.2:  the light bulb#7-10, light bulb#8-10 , bulb#9-10   [30QQTY5GMKZ9ES6DB76RSS8NBCCU7A]
    if not elems or len(elems)==1 or '#' not in elems[0]: 
        return ""
    confusing = [elem_from_countmap(elem) for elem in elems]
    return longest_overlapping_span(confusing)

def high_agreement(a_group, role_list, min_agreement):
    # Input: us cool#12-14, keeps us cool#11-14, us cool#12-14
    # Output: us cool#12-14
    # Step 1: choose clear winner.
    # Step 2: otherwise, increase span and choose winner.
    #    case 2.1:  the bulb#8-10, , bulb#9-10  [30QQTY5GMKZ9ES6DB76RSS8NBCCU7A]
    #    case 2.2:  the light bulb#7-10, light bulb#8-10 , bulb#9-10   [30QQTY5GMKZ9ES6DB76RSS8NBCCU7A]
    result = []
    for attr in role_list:
            v = a_group[attr].value_counts()  # phrase frequency/ agreement within turkers for a slot value.
            result_temp_list = [elem_from_countmap(r1) 
                           for r1 in str(v[v>=min_agreement]).split('\n') 
                           if ':' not in r1 and len(r1)>1]
            if result_temp_list:
                result.append(''.join(result_temp_list))
            elif len(v[v>=min_agreement])==0:
                v_elems = [v_elem for v_elem in str(v[v>=1]).split('\n')                            
                                                   if ':' not in v_elem and len(v_elem)>1]
                result.append(high_agreement_during_tie(v_elems))
            else:
                result.append("")
    return '\t'.join(result)

# see https://stackoverflow.com/questions/17071871/select-rows-from-a-dataframe-based-on-values-in-a-column-in-pandas
def highconf_data_all(min_agreement=2):
    df = pd.read_csv(outfile, sep='\t')
    grouped = df.groupby(['hit_id', 'verbid'])
    role_list=['assignment_id','hit_id','worker_id','sentid','senttokens','verbid','verb', 'who',
               'what', 'when','where','whereFrom','whereTo','input','output','enable','prevent']
    out_handle = open(outfile_highconf, 'w')
    out_handle.write('\t'.join(role_list) +"\n")
    for name,a_group in tqdm(grouped):
        try:
#         if(name[0]=='30QQTY5GMKZ9ES6DB76RSS8NBCCU7A'):
            out_handle.write(high_agreement(a_group, role_list, min_agreement) +"\n")
        except:
            print "Exception in assignment: ", name
    print "Output is in ", outfile_highconf
highconf_data_all()

100%|██████████| 4116/4116 [02:19<00:00, 26.46it/s]

Output is in  /Users/nikett/quick/sciencesrl-annotation-afresh/results/batch_v5_3__round3.highconf.tsv


# load and basic processing

In [ ]:
# import pandas as pd
# from collections import Counter, defaultdict
# import json
# from mturk_utils.mturk import pickle_this, unpickle_this

# def create_result(assmt):
#     result = {}
#     raw_result = json.loads(assmt.answers[0][0].fields[0])
#     result['image_id'] = raw_result['image_url']
#     result['object_words_raw'] = sorted([w.replace('_0_','_').replace('None', 'None_0_0') for w in raw_result['description']], key=lambda x: ''.join(x.split('_')[1:]))
#     obj_words = [word.split('_')[0] for word in result['object_words_raw']]
#     obj_word_location = [word.split('_')[1:] for word in result['object_words_raw']]
#     result['object_words'] = obj_words
#     result['object_locs'] = obj_word_location
#     result['asgmt_id'] = assmt.AssignmentId
#     result['hit_id'] = assmt.HITId
#     result['worker_id'] = assmt.WorkerId
#     return result

# turk_data = unpickle_this(infile)
# print turk_data
# assignments = [item for sublist in turk_data.values() for item in sublist]
# assignment_results = [create_result(ar) for ar in assignments]

# analysis

In [ ]:
def overlap(row):
    for idx in range(row.shape[0]):
        if row[idx] == None:
            row[idx] = set([])
    try:
        return list(row[0].intersection(row[1]).union(row[1].intersection(row[2])).union(row[0].intersection(row[2])))
    except TypeError:
        return {}

def distill_objects(obj_idx_list):
    combined_objects = []
    object_coords = []
    
    for word_position in obj_idx_list:
        split_components = word_position.split('_')
        word, sent_n, word_n = split_components[0], int(split_components[1]), int(split_components[2])
        object_coords.append((word, sent_n, word_n))
    object_coords = sorted(object_coords, key= lambda x: (x[1], x[2]))
    combined_objects.append(object_coords[0])
    
    for idx in range(1, len(object_coords)):
        this_word, this_sent, this_wn = object_coords[idx]
        last_word, last_sent, last_wn = combined_objects[-1]
        if this_sent == last_sent and last_wn + 1 == this_wn:            
            combined_objects[-1] = (' '.join([last_word, this_word]), this_sent, this_wn)
        else:
            combined_objects.append((this_word, this_sent, this_wn))
    return [w[1:] for w in combined_objects], [w[0] for w in combined_objects]

# Generates spans from words with word ids in text.
# e.g., words_input: [u'change_10', u'stay_23', u'the_24', u'same_25', u'may_9']
# returns: [u'may_9 change_10', u'stay_23 the_24 same_25']
def spans_from(words_input):
    # step1: sort words by position
    # u'change_10', u'stay_23', u'may_9' => u'may_9', u'change_10', u'stay_23'
    sorted_words = words_input
    sorted_words.sort(key=lambda x: int(x.split('_')[1]))
    # e.g., u'may_9', u'change_10', u'stay_23' (being sorted now)
    spans_from_words = ""
    for i in range(0,len(sorted_words)):
        prev_word_position = i-1 if(i>0) else 0
        # extract word index (e.g., 23 from stay_23) to compare adjacent (sorted) words
        curr_index = (int(sorted_words[i].split('_')[1]))
        prev_index = (int(sorted_words[prev_word_position].split('_')[1]))
        # insert comma at span boundaries.
        spans_from_words += ("," if(curr_index - prev_index >1 and i>0) else " ") + (sorted_words[i])
    return spans_from_words.strip().split(',')

st4_df = pd.DataFrame(assignment_results)
spans = [spans_from(st4_df['object_words_raw'][i]) for i in range(0, st4_df['object_words_raw'].count())]
# insert spans as the last column.
st4_df['spans'] = spans
st4_df.to_csv(outfile, sep='\t')
print "Result in " + outfile

In [ ]:
# Distill good workers.
# Find turkers who had proposed "None" for most of the hits where others didn't.
# st4_df[st4_df['worker_id']=='A3EAOSOUQZFVFS']
st4_df[st4_df['worker_id']=='A3EAOSOUQZFVFS']
# sorted(st4_df[st4_df['object_words'].apply(lambda x: x[0]) == 'None']['worker_id'])

df = st4_df[['image_id', 'spans']]
df.to_csv('/tmp/t.tsv', sep='\t')
# my_dict={
# 'dummy':{'w':0} # don't know how else to initialize.
# }
# # my_dict.update({'item4': {'a':2, 'b':3}, 'item5':  {'a':20, 'b':30}})
# for s in df[df['image_id']=='7']['spans']:
#     my_dict['7'][s]= 1
# my_dict


# st4_df[['image_id', 'spans']].groupby('image_id').agg(lambda x: ';;'.join(set(x)))
# TODO: per image id, aggregate all spans that have support of two or more.
# TODO: check against manual gold.
#.groupby('image_id').agg(lambda x: ';;'.join(set(x)))

# Code in scala and manual filtering is here: /Users/nikett/Documents/work/code/kb-organization/extraction/aristoKB/src/main/scala/eventsAndProcesses/to_be_fixed/predicateid/PredicateIDTurked.scala



sentence_annotations = defaultdict(lambda: defaultdict(int))

for i, id in enumerate(sendid):
    sentence_annnotations[id][annotation] += 1

# filtering step
# using dict comprehension here doesn't look good and gets a little complicated

for key, value in sentence_annotations.keys():
     sentence_annotation[key] = dict((key, value) for key, value in sent_annotation[key].items() if value > 1)